In [17]:
from trips.testProblems import *
from pylops import Identity, LinearOperator
from trips.utils import operator_qr, operator_svd, is_identity
from trips.parameter_selection.gcv import *
from trips.parameter_selection.discrepancy_principle import *
import scipy.linalg as la
import scipy.optimize as op
import numpy as np
# from trips.solvers.Tikhonov import *
# from trips.solvers.Hybrid_LSQR import *

In [18]:
D1D = Deblurring1D()
N = 64
n = N
A = D1D.forward_Op_matrix_1D(3, N)
# A = np.vstack((A, np.random.rand(3,N)))
x_true = D1D.gen_xtrue(N, test = 'piecewise')
b_true = A@x_true
(b, delta) = D1D.add_noise(b_true, 'Gaussian', 0.05)
# plt.plot(x_true)
# D1D.plot_data(b)

In [19]:
deltavar1 = la.norm(b.reshape((-1,1)) - (A@x_true).reshape((-1,1)))
deltavar2 = la.norm(b.reshape((-1,1)) - np.matmul(A,x_true))
# deltavar = la.norm(b.reshape((-1,1)) - (A@x_true))
print(deltavar1)
print(deltavar2)
print(np.matmul(A,x_true).shape)
deltavar = deltavar1

0.06737815009758147
11.02777859789314
(64,)


In [20]:
import numpy as np 
import scipy.linalg as la

from trips.utils import operator_qr, operator_svd, is_identity
import warnings

def discrepancy_principle_new(A, b, L, delta = None, eta = 1.01, **kwargs):

    if not ( isinstance(delta, float) or isinstance(delta, int)):

        # raise TypeError('You must provide a value for the noise level delta.')
        raise Exception("""A value for the noise level delta was not provided and the discrepancy principle cannot be applied. 
                    Please supply a value of delta based on the estimated noise level of the problem, or choose the regularization parameter according to gcv.""")

    if is_identity(L):
        Anew = A
        bnew = b
    else: ### MORE CASES TO BE CONSIDERED
        UL, SL, VL = la.svd(L)
        if L.shape[0] >= L.shape[1] and SL[-1] != 0:
            print('here')
            Anew = A@(VL.T@np.diag((SL)**(-1)))
            bnew = b
        elif L.shape[0] >= L.shape[1] and SL[-1] != 0:
            zeroind = np.where(SL == 0)
            W = VL[:,zeroind]
            AW = A@W
            Q_AW, R_AW = np.linalg.qr(AW, mode='reduced')
            Q_LT, R_LT = np.linalg.qr(L.T, mode='reduced')
            LAwpinv = (np.eye(L.shape[1]) - (W@np.linalg.inv(R_AW)@Q_AW.T@A))@Q_LT@np.linalg.inv(R_LT.T)
            Anew = A@LAwpinv
            xnull = W@np.linalg.inv(R_AW)@Q_AW.T@b
            bnew = b - A@xnull
        elif  (L.shape[0] < L.shape[1]):
            print(L.shape[0]-L.shape[1])
            W = VL[:,L.shape[0]-L.shape[1]:]
            # print(W.shape)
            AW = A@W
            Q_AW, R_AW = np.linalg.qr(AW, mode='reduced')
            Q_LT, R_LT = np.linalg.qr(L.T, mode='reduced')
            print((W@np.linalg.inv(R_AW)@Q_AW.T@A).shape)
            LAwpinv = (np.eye(L.shape[1]) - (W@np.linalg.inv(R_AW)@Q_AW.T@A))@Q_LT@np.linalg.inv(R_LT.T)
            Anew = A@LAwpinv
            xnull = W@np.linalg.inv(R_AW)@Q_AW.T@b
            bnew = b - A@xnull

    
    U, S, V = la.svd(Anew)
    singular_values = S**2
    if Anew.shape[0] > Anew.shape[1]:
        singular_values = np.append(singular_values.reshape((-1,1)), np.zeros((Anew.shape[0]-Anew.shape[1],1)))
    singular_values.shape = (singular_values.shape[0], 1)
    bhat = U.T @ bnew
    
    beta = 1e-8
    iterations = 0

    while (iterations < 30) or ((iterations <= 100) and (np.abs(alpha) < 10**(-16))):
        print(iterations)
        zbeta = (((singular_values*beta + 1)**(-1))*bhat.reshape((-1,1))).reshape((-1,1))
        f = la.norm(zbeta)**2 - (eta*delta)**2
        wbeta = (((singular_values*beta + 1)**(-1))*zbeta).reshape((-1,1))
        f_prime = 2/beta*zbeta.T@(wbeta - zbeta)

        # tikh_sol = lambda reg_param: np.linalg.lstsq(np.vstack((Anew, (1/np.sqrt(reg_param))*np.eye(Anew.shape[1]))), np.vstack((bnew.reshape((-1,1)), np.zeros((Anew.shape[1],1)))))[0]
        # discr_func_zero = lambda reg_param: (np.linalg.norm(np.matmul(Anew,tikh_sol(reg_param)).reshape((-1,1)) - (bnew.reshape((-1,1))))**2 - (eta*delta)**2)
        tikh_sol = lambda reg_param: np.linalg.lstsq(np.vstack((A, (1/np.sqrt(reg_param))*L)), np.vstack((bnew.reshape((-1,1)), np.zeros((L.shape[0],1)))))[0]
        discr_func_zero = lambda reg_param: (np.linalg.norm(np.matmul(A,tikh_sol(reg_param)).reshape((-1,1)) - (b.reshape((-1,1))))**2 - (eta*delta)**2)

        print(f)
        print(discr_func_zero(beta))

        beta_new = beta - f/f_prime

        if abs(beta_new - beta) < 10**(-12)* beta:
            break

        beta = beta_new
        alpha = 1/beta_new

        iterations += 1


    return alpha#{'x':alpha}

In [21]:
outTikhLvar = Tikhonov(A, b, Lvar, x_true, regparam = 'dp', delta = deltavar)
xTikhLvar = outTikhLvar[0]
alphaLvar = outTikhLvar[1]
print(alphaLvar)
print(la.norm((A@xTikhLvar).reshape((-1,1)) - b.reshape((-1,1))) - 1.01*deltavar)

TypeError: 'module' object is not callable

In [22]:
import numpy as np 
from scipy.optimize import newton, minimize
import scipy.optimize as op
import scipy.linalg as la

from trips.utils import operator_qr, operator_svd, is_identity
# from .gcv import gcv_numerator
import warnings

def discrepancy_principle_basic(A, b, L, delta = None, eta = 1.01, **kwargs):

    if not ( isinstance(delta, float) or isinstance(delta, int)):

        # raise TypeError('You must provide a value for the noise level delta.')
        raise Warning("""A value for the noise level delta was not provided. A default value of 0.01 has been used. 
                    Please supply a value of delta based on the estimated noise level of the problem.""")

        delta = 0.01
    
    tikh_sol = lambda reg_param: np.linalg.lstsq(np.vstack((A, np.sqrt(reg_param)*L)), np.vstack((b.reshape((-1,1)), np.zeros((L.shape[0],1)))))[0]
    discr_func_zero = lambda reg_param: np.linalg.norm(np.matmul(A,tikh_sol(reg_param)).reshape((-1,1)) - b.reshape((-1,1))) - (eta*delta)
    alpha = op.fsolve(discr_func_zero, 1e-10)[0]

    return alpha#{'x':alpha}

In [23]:
import numpy as np
from trips.decompositions import golub_kahan, arnoldi
from trips.parameter_selection.gcv import generalized_crossvalidation
# from trips.parameter_selection.discrepancy_principle import discrepancy_principle
from trips.utils import smoothed_holder_weights
from collections.abc import Iterable
def Tikhonov(A, b, L, x_true, regparam = 'gcv', **kwargs):
    if regparam in ['gcv', 'GCV', 'Gcv']:
        lambdah = generalized_crossvalidation(A, b, L) # find ideal lambda by crossvalidation ###
        xTikh = np.linalg.solve(A.T@A + lambdah*L.T@L, A.T@b)
    elif regparam in ['DP', 'dp', 'Dp', 'Discrepancy Principle', 'Discrepancy principle', 'discrepancy principle']:
        lambdah = discrepancy_principle_new(A, b, L, **kwargs) # find ideal lambdas by discrepancy principle
        # print(lambdah)
        xTikh = np.linalg.solve(A.T@A + lambdah*L.T@L, A.T@b)
    else:
        lambdah = regparam
        xTikh = np.linalg.solve(A.T@A + lambdah*L.T@L, A.T@b)
    return xTikh, lambdah  

In [24]:
L = np.eye(N)
outTikhI = Tikhonov(A, b, L, x_true, regparam = 'dp', delta = deltavar)
xTikhI = outTikhI[0]
alphaI = outTikhI[1]
print(alphaI)
print(la.norm((A@xTikhI).reshape((-1,1)) - b.reshape((-1,1)))-1.01*deltavar)

0
1.8407549167422623
1.8407549167422605
1
0.8700988264194941
0.8700988264194925
2
0.41228557103514846
0.4122855710351476
3
0.19529213820441668
0.19529213820441693
4
0.09231282786549914
0.09231282786549887
5
0.04367170342176509
0.04367170342176493
6
0.0207543142498483
0.020754314249848364
7
0.009846771243749164
0.00984677124374917
8
0.004557451202366963
0.00455745120236701
9
0.001961658238851966
0.0019616582388519927
10
0.0007124105271848107
0.0007124105271848471
11
0.00017462778356873345
0.0001746277835687508
12
1.6671529866875423e-05
1.667152986690578e-05
13
1.8377314836993258e-07
1.837731484080965e-07
14
2.2829506514387976e-11
2.2829546413027924e-11
15
0.0
4.5102810375396984e-17
[[0.00331115]]
-4.163336342344337e-17


<ipython-input-20-0ca58d879f53>:67: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  tikh_sol = lambda reg_param: np.linalg.lstsq(np.vstack((A, (1/np.sqrt(reg_param))*L)), np.vstack((bnew.reshape((-1,1)), np.zeros((L.shape[0],1)))))[0]


In [25]:
diags = [-np.ones((N,)), np.ones((N-1,))] 
L = sparse.diags(diags, [0, 1]).toarray()
rank = np.linalg.matrix_rank(L)
print(rank)

64


In [26]:
outTikhL = Tikhonov(A, b, L, x_true, regparam = 'dp', delta = deltavar)
xTikhL = outTikhL[0]
alphaL = outTikhL[1]
print(alphaL)
print(la.norm((A@xTikhL).reshape((-1,1)) - b.reshape((-1,1))) - 1.01*deltavar)

here
0
1.8407281159052458
1.8407281159052447
1
1.1639613817831778
1.1639613817831758
2
0.8637693477235228
0.863769347723522
3
0.5708590657746069
0.5708590657746069
4
0.32486520126566404
0.3248652012656643
5
0.16530360131213676
0.16530360131213676
6
0.08330578576318047
0.08330578576318066
7
0.04155705609682999
0.04155705609682983
8
0.02114549320768687
0.021145493207686737
9
0.010945107584983859
0.010945107584983817
10
0.005493281589578502
0.005493281589578494
11
0.0025527402914060854
0.002552740291406055
12
0.0010331807118630007
0.0010331807118630215
13
0.0003156214358397231
0.00031562143583972746
14
5.0451117667985566e-05
5.0451117667969954e-05
15
1.753355676292409e-06
1.7533556762906743e-06
16
2.2678021769501067e-09
2.2678021769501067e-09
17
3.804248582817138e-15
3.788636071533347e-15
18
0.0
1.734723475976807e-18
[[0.00668714]]
-1.3877787807814457e-17


<ipython-input-20-0ca58d879f53>:67: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  tikh_sol = lambda reg_param: np.linalg.lstsq(np.vstack((A, (1/np.sqrt(reg_param))*L)), np.vstack((bnew.reshape((-1,1)), np.zeros((L.shape[0],1)))))[0]


In [27]:
print(alphaI - alphaL)

[[-0.00337599]]


In [28]:
diags = [-np.ones((N,)), np.ones((N-1,))] 
L = sparse.diags(diags, [0, 1]).toarray()
Lvar = L[:-1,:]
print(L.shape)

(64, 64)


In [29]:
L = np.eye(N)
outTikhI = Tikhonov(A, b, L, x_true, regparam = 'dp', delta = deltavar)
xTikhI = outTikhI[0]
alpha = outTikhI[1]
print(alpha)
print(la.norm((A@xTikhI).reshape((-1,1)) - b.reshape((-1,1)))-1.01*deltavar)

0
1.8407549167422623
1.8407549167422605
1
0.8700988264194941
0.8700988264194925
2
0.41228557103514846
0.4122855710351476
3
0.19529213820441668
0.19529213820441693
4
0.09231282786549914
0.09231282786549887
5
0.04367170342176509
0.04367170342176493
6
0.0207543142498483
0.020754314249848364
7
0.009846771243749164
0.00984677124374917
8
0.004557451202366963
0.00455745120236701
9
0.001961658238851966
0.0019616582388519927
10
0.0007124105271848107
0.0007124105271848471
11
0.00017462778356873345
0.0001746277835687508
12
1.6671529866875423e-05
1.667152986690578e-05
13
1.8377314836993258e-07
1.837731484080965e-07
14
2.2829506514387976e-11
2.2829546413027924e-11
15
0.0
4.5102810375396984e-17
[[0.00331115]]
-4.163336342344337e-17


<ipython-input-20-0ca58d879f53>:67: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  tikh_sol = lambda reg_param: np.linalg.lstsq(np.vstack((A, (1/np.sqrt(reg_param))*L)), np.vstack((bnew.reshape((-1,1)), np.zeros((L.shape[0],1)))))[0]


In [14]:
from trips.decompositions import golub_kahan_update
import numpy as np
from scipy import linalg as la
from trips.parameter_selection.gcv import *
# from trips.parameter_selection.discrepancy_principle import *
from pylops import Identity
from trips.solvers import Tikhonov

def hybrid_lsqr(A, b, n_iter, regparam = 'gcv', **kwargs): # what's the naming convention here?

    delta = kwargs['delta'] if ('delta' in kwargs) else None

    eta = kwargs['eta'] if ('eta' in kwargs) else 1.01

    if regparam == 'dp' and delta == None:
        raise Exception("""A value for the noise level delta was not provided and the discrepancy principle cannot be applied. 
                    Please supply a value of delta based on the estimated noise level of the problem, or choose the regularization parameter according to gcv.""")
    
    n = A.shape[1]
    beta = np.linalg.norm(b)
    U = b.reshape((-1,1))/beta
    B = np.empty(1)
    V = np.empty((n,1))
    RegParam = np.zeros(n_iter,)
    Rnrm = np.zeros(n_iter,)

    for ii in range(n_iter):
        # print(ii)
        (U, B, V) = golub_kahan_update(A, U, B, V)
        bhat = np.zeros(ii+2,); bhat[0] = beta ###
        L = Identity(B.shape[1], B.shape[1])
        y = la.lstsq(B,bhat)[0]
        nrmr = np.linalg.norm(bhat - B@y)
        Rnrm[ii] = nrmr
        if ii == 0:
            lambdah = 0
        else:
            if regparam == 'gcv':
                #lambdah = generalized_crossvalidation(B, bhat, L, **kwargs)['x'].item()
                lambdah = generalized_crossvalidation(B, bhat, L, **kwargs)
            elif regparam == 'dp':
                print(ii)
                # print(eta)
                # print(delta)
                # print(nrmr)
                if nrmr <= eta*delta:
                    lambdah = discrepancy_principle_new(B, bhat, L, **kwargs)#['x'].item()
                    # lambdah = 0
                else:
                    lambdah = 0
            else:
                lambdah = regparam
            RegParam[ii] = lambdah
            print(lambdah)
            L = L.todense() if isinstance(L, LinearOperator) else L
            y = la.lstsq(np.matmul(B.T,B) + lambdah*np.matmul(L.T,L), np.matmul(B.T,bhat))[0] ## SG: potentially needs L.T*L; also why lstsq? May just need a solver for linear system; also: you may need np.matmul; also: these are not the normal eqs (you need B'*b)
            print(la.norm(B@y - bhat) - eta*delta)
            #y = Tikhonov(B, bhat, L, regparam = lambdah)[0]
            x = V @ y
    return x, U, B, V, RegParam, Rnrm, L

In [16]:
out_hybridlsqr = hybrid_lsqr(A, b, 20, regparam = 'dp', delta = deltavar)

1
0
0.20436508131388573
2
0
0.1258798505843784
3
0
0.06411399002656887
4
0
0.0489715360629155
5
0
0.042199965726728864
6
0
0.027033146289538235
7
0
0.01996577661612209
8
0
0.016245987223089883
9
0
0.011862460170661684
10
0
0.00419257738768479
11
0
0.0005381715823420952
12
0
1.8013724351707616


ValueError: all the input array dimensions for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 13 and the array at index 1 has size 1